In [1]:
import salem 
import xarray as xr
import pandas as pd
#import xesmf as xe
import matplotlib.pyplot as plt
import glob
import numpy as np

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
ds=xr.open_dataset('/exports/d1/wrf_only_daily/fnl1.0/2019M01/2019010100_physical_0.nc')#.U[16,0,].values
ds

<xarray.Dataset>
Dimensions:   (Time: 24, south_north: 430, west_east: 600, bottom_top: 44,
               west_east_stag: 601, south_north_stag: 431, ncl7: 24, ncl8: 44,
               ncl9: 430, ncl10: 600)
Dimensions without coordinates: Time, south_north, west_east, bottom_top,
                                west_east_stag, south_north_stag, ncl7, ncl8,
                                ncl9, ncl10
Data variables: (12/26)
    Times     (Time) |S19 ...
    xlat      (south_north, west_east) float32 ...
    xlon      (south_north, west_east) float32 ...
    height    (Time, bottom_top, south_north, west_east) float32 ...
    pressure  (Time, bottom_top, south_north, west_east) float32 ...
    pblh      (Time, south_north, west_east) float32 ...
    ...        ...
    wdir      (Time, bottom_top, south_north, west_east) float32 ...
    wspd      (Time, bottom_top, south_north, west_east) float32 ...
    wd10      (Time, south_north, west_east) float32 ...
    ws10      (Time, south_north, west_east) float32 ...
    dens      (ncl7, ncl8, ncl9, ncl10) float32 ...
    psfc      (Time, south_north, west_east) float32 ...

In [3]:
'''30.774N, 90.988E，这个经纬度
2019年5月1日到7月9日
早晨8点到晚上20点
高度200米范围内
'''

'30.774N, 90.988E，这个经纬度\n2019年5月1日到7月9日\n早晨8点到晚上20点\n高度200米范围内\n'

In [4]:
def nearest_position( stn_lat, stn_lon, lat2d, lon2d):
    """获取最临近格点坐标索引
    stn_lat  : 站点纬度
    stn_lon  : 站点经度
    lat2d    : numpy.ndarray网格二维经度坐标
    lon2d    : numpy.ndarray网格二维纬度坐标
    Return: (y_index, x_index)
    """
    difflat = stn_lat - lat2d;
    difflon = stn_lon - lon2d;
    rad = np.multiply(difflat,difflat)+np.multiply(difflon , difflon)
    aa=np.where(rad==np.min(rad))
    ind=np.squeeze(np.array(aa))
    return tuple(ind)
lat2d=ds.xlat
lon2d=ds.xlon
stn_lat=30.774
stn_lon=90.988
id1,id2=nearest_position( stn_lat, stn_lon, lat2d, lon2d)

In [7]:
ds.xlon[id1,id2]

<xarray.DataArray 'xlon' ()>
array(90.98419, dtype=float32)
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  LONGITUDE, WEST IS NEGATIVE
    units:        degree_east
    stagger:      
    coordinates:  XLONG XLAT

In [20]:
ds.height[0,:,id1,id2].load().values.astype(int)<200

array([ True,  True,  True,  True,  True,  True,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

In [8]:
ds.xlat[id1,id2]

<xarray.DataArray 'xlat' ()>
array(30.807167, dtype=float32)
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  LATITUDE, SOUTH IS NEGATIVE
    units:        degree_north
    stagger:      
    coordinates:  XLONG XLAT

In [43]:


dssel=ds[['Times','u','v','height','xlon','xlat','bottom_top']].sel(south_north=id1,west_east=id2,bottom_top=np.arange(7))

# 将字节串转换为普通字符串
str_array = dssel['Times'].values.astype(str)

# 将字符串转换为datetime
datetime_array = pd.to_datetime(str_array, format='%Y-%m-%d_%H:%M:%S')
#datetime_array

dssel['Time']=datetime_array+pd.Timedelta('8H')#=pd.date_range('20220821T00','20220821T23',freq='1H')+pd.Timedelta('8H')
dssel#['Time']

dssel=dssel.sel(Time=dssel.Time.dt.hour.isin(np.arange(8,21,1)))
dssel['wind']=np.sqrt(dssel.u**2+dssel.v**2)
dssel

<xarray.Dataset>
Dimensions:     (Time: 13, bottom_top: 7)
Coordinates:
  * bottom_top  (bottom_top) int64 0 1 2 3 4 5 6
  * Time        (Time) datetime64[ns] 2019-01-01T08:00:00 ... 2019-01-01T20:0...
Data variables:
    Times       (Time) |S19 b'2019-01-01_00:00:00' ... b'2019-01-01_12:00:00'
    u           (Time, bottom_top) float32 0.5531 0.8811 1.041 ... 3.903 4.16
    v           (Time, bottom_top) float32 1.464 1.02 0.8258 ... -0.2543 -0.4743
    height      (Time, bottom_top) float32 ...
    xlon        float32 90.98
    xlat        float32 30.81
    wind        (Time, bottom_top) float32 1.565 1.348 1.328 ... 3.911 4.187

In [40]:
L=glob.glob('/exports/d1/wrf_only_daily/fnl1.0/2019M0[5-7]/*_physical_0.nc')
L.sort()
L

['/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050100_physical_0.nc',
 '/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050200_physical_0.nc',
 '/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050300_physical_0.nc',
 '/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050400_physical_0.nc',
 '/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050500_physical_0.nc',
 '/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050600_physical_0.nc',
 '/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050700_physical_0.nc',
 '/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050800_physical_0.nc',
 '/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050900_physical_0.nc',
 '/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019051000_physical_0.nc',
 '/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019051100_physical_0.nc',
 '/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019051200_physical_0.nc',
 '/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019051300_physical_0.nc',
 '/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019051400_physical_0.nc',
 '/exp

In [44]:
DS=[]
for l in L:
    ds=xr.open_dataset(l)
    
    

    dssel=ds[['Times','u','v','height','xlon','xlat','bottom_top']].sel(south_north=id1,west_east=id2,bottom_top=np.arange(7))

    # 将字节串转换为普通字符串
    str_array = dssel['Times'].values.astype(str)

    # 将字符串转换为datetime
    datetime_array = pd.to_datetime(str_array, format='%Y-%m-%d_%H:%M:%S')
    #datetime_array

    dssel['Time']=datetime_array+pd.Timedelta('8H')#=pd.date_range('20220821T00','20220821T23',freq='1H')+pd.Timedelta('8H')
    dssel#['Time']

    dssel=dssel.sel(Time=dssel.Time.dt.hour.isin(np.arange(8,21,1)))
    dssel['wind']=np.sqrt(dssel.u**2+dssel.v**2)
    dssel.load()
    
    DS.append(dssel)
    print(l)

/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050100_physical_0.nc
/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050200_physical_0.nc
/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050300_physical_0.nc
/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050400_physical_0.nc
/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050500_physical_0.nc
/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050600_physical_0.nc
/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050700_physical_0.nc
/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050800_physical_0.nc
/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019050900_physical_0.nc
/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019051000_physical_0.nc
/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019051100_physical_0.nc
/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019051200_physical_0.nc
/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019051300_physical_0.nc
/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019051400_physical_0.nc
/exports/d1/wrf_only_daily/fnl1.0/2019M05/2019051500_physical_

In [45]:
DS2=xr.concat(DS,dim='Time')

In [48]:
DS2.to_dataframe().reset_index().to_csv('wind.csv')